Что делать?
1.Датасет ml-latest
2.Вспомнить подходы, которые мы разбирали
3.Выбрать понравившийся подход к гибридным системам
4.Написать свою

In [34]:
from surprise import KNNWithMeans, KNNBasic, SVD, BaselineOnly
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
import pandas as pd
import numpy as np
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

In [35]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [36]:
data = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
data.dropna(inplace=True)

In [37]:
# задаем датасет в формате surprise
dataset = pd.DataFrame({
    'uid': data.userId,
    'iid': data.title,
    'rating': data.rating
})

In [38]:
reader = Reader(rating_scale=(0.5, 5.0))
data_surprise = Dataset.load_from_df(dataset, reader)

In [39]:
trainset, testset = train_test_split(data_surprise, test_size=.15, random_state=42)

In [40]:
# обучаем модель 
algo = SVD(n_factors=20, n_epochs=20)
algo.fit(trainset)

In [41]:
# делаем оценку качества на тесте
test_pred = algo.test(testset)

accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8672


0.8672364938885339

In [42]:
# оценка на кросс валидации
cros_val = cross_validate(algo, data_surprise, measures=['RMSE'], cv=5, verbose=True)
RMSE_mean = cros_val['test_rmse'].mean()

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8719  0.8728  0.8742  0.8685  0.8641  0.8703  0.0036  
Fit time          3.93    3.69    3.67    3.63    3.57    3.70    0.12    
Test time         0.44    0.25    0.32    0.24    0.23    0.30    0.08    


In [43]:
print(RMSE_mean)

0.8702814049741241


In [44]:
# определяем пользователя и фильмы которые он посмотрел
user = 7.0
user_movies = data[data.userId == user].title.unique()

In [45]:
# формируем списки по предсказанию и наименовнию фильма
pred_rating = []
titles = []

for movie in data.title.unique():
    if movie in user_movies:
        continue
        
    pred_rating.append(algo.predict(uid=user, iid=movie).est)
    titles.append(movie)

In [46]:
# база фильмов которые пользователь не смотрел и прогнозный рейтинг фильму от пользователя
movies_not_watch = pd.DataFrame({'title': titles, 'pred_r': pred_rating})

In [47]:
# берем топ 10 фильмом из потенциального прогноза пользователя
most_interesting = movies_not_watch.sort_values(by='pred_r', ascending=False)[:10]

In [48]:
# формируем базу фильмов с средним рейтингом по фактическим оценкам пользователя
movies_with_mean_rating = pd.DataFrame(data.groupby('title').rating.mean()).reset_index()

In [49]:
# задаем функцию которая добавит средний рейтинг
def add_mean(row):
    return movies_with_mean_rating[movies_with_mean_rating.title == row].rating.sum()

In [50]:
# в понтенциальные топ 10 добавляем фактический средний рейтинг
most_interesting['mean_rating'] = most_interesting.title.apply(add_mean)

In [51]:
# создаем метрику которая взвешивает два подхода на 70/30 процентов
most_interesting['common_rating'] = most_interesting.pred_r*0.7 + most_interesting.mean_rating*0.3

In [52]:
#формируем топ 3 рекомендации, на основе финальной метрики
recommend_for_user = most_interesting.sort_values(by='common_rating', ascending=False)[:3][['title', 'common_rating']]

In [54]:
recommend_for_user

,title,common_rating
1894,"Matrix, The (1999)",4.182613
893,"Godfather: Part II, The (1974)",4.075454
874,"Princess Bride, The (1987)",4.045295
